In [1]:
import torch
from transformers import pipeline
import sys
from datasets import load_dataset, Dataset, DatasetDict


/Users/davidgaofc/PycharmProjects/RLAIF/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import tensorflow as tf
print(tf.__version__)  # For TensorFlow

import torch
print(torch.__version__)

2.15.0
2.1.1


In [3]:
model_1 = "davidgaofc/SFT_Med_t"
model_2 = "davidgaofc/PPO_base"
model_3 = "davidgaofc/PPO_prima"
generator_sft = pipeline('text2text-generation', model=model_1)
generator_base = pipeline('text2text-generation', model=model_2)
generator_prima = pipeline('text2text-generation', model=model_3)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of the model checkpoint at davidgaofc/PPO_base were not used when initializing T5ForConditionalGeneration: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at davidgaofc/PPO_prima were not used when initializing T5ForConditionalGeneration: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

In [4]:
dataset_name = "davidgaofc/MedQuad_split"
dataset = load_dataset(dataset_name)

In [5]:
import random
def get_responses(prompt, generator1, generator2, generator3, batch_size=8, max_length=50):
    responses_1 = []
    responses_2 = []
    responses_3 = []
    counter = 1
    for i in range(0, len(prompt), batch_size):
        print(counter)
        batch_prompts = prompt[i:i + batch_size]

        # Generating the first set of responses
        batch_responses_1 = generator1(batch_prompts, max_length=max_length)
        # Generating the second set of responses
        batch_responses_2 = generator2(batch_prompts, max_length=max_length)
        # Generating the third set of responses
        batch_responses_3 = generator3(batch_prompts, max_length=max_length)


        responses_1.extend([response['generated_text'].strip() for response in batch_responses_1])
        responses_2.extend([response['generated_text'].strip() for response in batch_responses_2])
        responses_3.extend([response['generated_text'].strip() for response in batch_responses_3])
        counter += 1

    return list(zip(responses_1, responses_2, responses_3))

In [6]:
prompts = [entry["Question"] for entry in dataset["test"]]

batch_responses = get_responses(prompts, generator1=generator_sft, generator2=generator_base, generator3=generator_prima, batch_size=8, max_length=50)

records = []

for prompt, (response1,response2,response3) in zip(prompts, batch_responses):
  formatted_question = f"human: {prompt}\n"
  records.append({"Question": formatted_question, "SFT": response1, "Base_PPO": response2, "Prima_PPO": response3})

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


In [7]:
import pandas as pd
df = pd.DataFrame(records)
df.to_csv("pairwise_setup.csv", index=False)

In [8]:
new_dataset = Dataset.from_csv("pairwise_setup.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
new_dataset.push_to_hub("davidgaofc/pairwise_setup")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]